# Zomato Scrapping and Recommendation Engine Assignment

### Name: Sai Aditya Garlapati
### User ID: 18125760054

In [59]:
import surprise
from surprise import KNNWithZScore
from surprise import Reader, Dataset
import pandas as pd
import numpy as np
import bs4
import selenium
import requests
import re
from selenium import webdriver

In [47]:
browser = webdriver.Chrome('chromedriver.exe')


In [48]:
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [49]:
res =browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(res))

141


In [55]:
restaurents = pd.DataFrame(columns = ['Name', 'ID', 'URL'])
restaurent_names = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')
for restaurent in restaurent_names:
    name = restaurent.find_element_by_css_selector('div.res_title').text
    id = int(restaurent.get_attribute('data-entity-id'))
    url = restaurent.find_element_by_css_selector('div.ptop0.pbot0.pl10.pr10').find_element_by_tag_name('a').get_attribute('href')
    restaurents = restaurents.append({'Name':name, 'ID':id, 'URL':url},
                                     ignore_index = True)
restaurents.to_csv("restuarants.csv")

In [ ]:
df_reviews=pd.DataFrame(columns={'rating','user_name','user_id','rest_name'})
#df_reviews.to_csv('zomato_scrap.csv',index=False)
for link in df_rest['restaurant_link']:
    browser.get(link )
    while True:
        more_sel = 'div.load-more'
        try:
            load_more_button = browser.find_element_by_css_selector(more_sel)
            load_more_button.click()

            reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        except Exception:
            break
    def get_info(review, selector , attribute):
        try:    
            tag = review.find_element_by_css_selector(selector)

            tag = tag.get_attribute(attribute)
        except:
            tag=None
        return tag
        
    for review in reviews:  
            rating=get_info(review,'div.ttupper','aria-label')
            #y=re.findall('[0-9.]+',rating)
            #rating = float(y[0])
            name_selec = 'a[data-entity_id]'
            user_id =get_info(review,name_selec,'data-entity_id')  
            user_name = get_info(review,name_selec,'innerHTML')
            if user_name !=None:
                user_name = user_name.strip()
            rest_name = get_info(browser,'a.ui.large.header','title') 
            curr_review={
                         'User_Name':user_name,
                         'Rating':rating,
                         'User_ID':user_id,
                         'Restaurent_Name':rest_name
                     }
            df_reviews= df_reviews.append(curr_review,ignore_index=True)
#df_reviews.head()  

### scrapping code already ran once so i am skipping this step during restarting kernel

In [32]:
df_reviews.head() 

,rating,user_id,user_name,rest_name
0,Rated 4.0,16214943,Khushbu Rithal,Avista Cafe Pizzeria Bangalore
1,Rated 5.0,7225951,Vishnu Das,Avista Cafe Pizzeria Bangalore
2,Rated 5.0,501410,Saunak Mukherjee,Avista Cafe Pizzeria Bangalore
3,Rated 4.0,27258850,Abhinaya,Avista Cafe Pizzeria Bangalore
4,Rated 4.0,83298879,ROSHNI DESHMUKH,Avista Cafe Pizzeria Bangalore


In [7]:
df_reviews.shape[0]

14373

In [8]:
df_reviews.to_csv('zomato_scrap.csv',index=False)

In [9]:
import pandas as pd

In [56]:
res =pd.read_csv("restuarants.csv")
res.head()

,Unnamed: 0,Name,ID,URL
0,0,Sea Lions BBQ & Grills,18767897,https://www.zomato.com/bangalore/sea-lions-bbq...
1,1,The Reservoire,18537585,https://www.zomato.com/bangalore/the-reservoir...
2,2,Market Table,18622268,https://www.zomato.com/bangalore/market-table-...
3,3,"3B's - Buddies, Bar &...",18202991,https://www.zomato.com/bangalore/3bs-buddies-b...
4,4,Alba - JW Marriott Bengaluru,55908,https://www.zomato.com/bangalore/alba-jw-marri...


In [60]:
ratings = pd.read_csv("zomato_scrap.csv")
ratings.head()

,Restaurent_Name,Restaurent_ID,User_Name,User_ID,Rating
0,Brewklyn Microbrewery,18426381,Isha Kaur,36363182,4.0
1,Brewklyn Microbrewery,18426381,Shreyas,27720568,4.0
2,Brewklyn Microbrewery,18426381,Aaghran Ghosh,245984,4.0
3,Brewklyn Microbrewery,18426381,Didon Clinton Dsouza,37440297,5.0
4,Brewklyn Microbrewery,18426381,Anant :),39373492,4.0


In [73]:
import pandas_profiling as pp

In [74]:
pp.ProfileReport(ratings)

Number of variables,5
Number of observations,6210
Total Missing (%),3.1%
Total size in memory,242.7 KiB
Average record size in memory,40.0 B
Numeric,3
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [67]:
ratings.tail()

,Restaurent_Name,Restaurent_ID,User_Name,User_ID,Rating
6205,The Blue Wagon - Kitchen,18701756,Varsegu,1309970,5.0
6206,The Blue Wagon - Kitchen,18701756,Rajat Khurana.,532189,5.0
6207,The Blue Wagon - Kitchen,18701756,Bengaluru Food Walks,719993,5.0
6208,The Blue Wagon - Kitchen,18701756,The Greedy Guts,20191609,5.0
6209,The Blue Wagon - Kitchen,18701756,Nikita Dugar,1302604,5.0


In [66]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(ratings[['User_ID','Restaurent_ID', 'Rating']], reader)
sim_options = {
    'name': 'cosine',
    'user_based': True,
    'k':10
}

knn = KNNWithZScore(sim_options = sim_options)
knn.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [68]:
knn.predict(uid = 20191609, iid = 18701756, verbose = True)

user: 20191609   item: 18701756   r_ui = None   est = 4.81   {'actual_k': 24, 'was_impossible': False}


Prediction(uid=20191609, iid=18701756, r_ui=None, est=4.811840851545116, details={'actual_k': 24, 'was_impossible': False})

In [69]:
def Prediction(userid):
    predictions = pd.DataFrame()

    for restaurentid in ratings['Restaurent_ID'].unique():
      restaurent_row = ratings[(ratings['User_ID'] == userid) & (ratings['Restaurent_ID'] == restaurentid)]
      if len(restaurent_row) == 0:
        restaurent_pred_rating = knn.predict(uid = userid, iid = restaurentid)[3]
        predictions = predictions.append({'Restaurent_Id':restaurentid, 'rating_prediction':restaurent_pred_rating}, ignore_index = True)
    ids = predictions.sort_values('rating_prediction', ascending = False).head(3)['Restaurent_Id'].astype('int')
    return restaurents.loc[ids]['Restaurent_Name']

In [70]:
Prediction(20191609)

Restaurent_ID
61111            Milano Ice Cream
52002                Veena Stores
58268    The Fatty Bao - Asian...
Name: Restaurent_Name, dtype: object